In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,UpSampling3D, Conv3DTranspose, Flatten, Concatenate, Dense, TimeDistributed, Bidirectional, Input, Reshape  
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
from time import sleep


In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=18000)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)


In [ ]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.debugging.set_log_device_placement(True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
def create_sequences(images, sequence_length):
    X = []
    num_sequences = len(images) // sequence_length
    for i in range(num_sequences):
        start_idx = i * sequence_length
        end_idx = start_idx + sequence_length
        sequence_x = images[start_idx:end_idx]
        X.append(sequence_x)

    return np.array(X)

In [ ]:
parquet_df =  pd.read_parquet("/home/arman_abouali/Downloads/X_data.parquet")
parquet_df = parquet_df.sort_values(by='Key', ascending=True)
parquet_df.reset_index(drop=True, inplace=True)

print(parquet_df)

In [ ]:
# Extract the 'Value' column
values = parquet_df['Value'].values

width = 71
length = 41

X_images = np.zeros(shape=(len(values), width, length))
for ind, val in enumerate(values):
    X_images[ind, :, :] = np.stack(val,axis=1)
X_images = np.where(X_images<0, 0, X_images)

# X += np.random.normal(loc=0, scale=1, size=X.shape)

y_df = pd.read_csv('test.csv', sep=';', parse_dates=['Zeit'])
y_df.set_index('Zeit', inplace=True)
y = y_df.loc['2016-08-01':'2017-08-31'][['Margarethenklippe_Pegel_now', 'Sennhuette_Pegel_now']].values
# y = merged_df[['Margarethenklippe','Sennhuette']].values

# Verify the shape of X
print(f"Shape of X: {X_images.shape}")
print(f"Shape of y: {y.shape}")


In [ ]:
scaler = MinMaxScaler()
y = scaler.fit_transform(y)
print(f"Shape of y_scaled: {y.shape}")

In [ ]:
seq_length = 4

X_image_sequence = create_sequences(X_images, seq_length)[:-1, :, :]

X_sensors_sequence = create_sequences(y, sequence_length=seq_length)

y_sensors_sequence = X_sensors_sequence[1:, -1, :]

X_sensors_sequence = X_sensors_sequence[:-1, :, :]

In [ ]:
train_len = int(X_image_sequence.shape[0]*0.8)
val_len = int(X_image_sequence.shape[0]*0.1)

X_im_train = X_image_sequence[:train_len]
X_im_val = X_image_sequence[train_len:train_len+val_len]
X_im_test = X_image_sequence[train_len+val_len:]

X_sen_train = X_sensors_sequence[:train_len]
X_sen_val = X_sensors_sequence[train_len:train_len+val_len]
X_sen_test = X_sensors_sequence[train_len+val_len:]

im_max = np.max(X_im_train)
sen_max = np.max(X_sen_train)


X_im_train /= im_max
X_im_val /= im_max
X_im_test /= im_max

X_sen_train /= sen_max
X_sen_val /= sen_max
X_sen_test /= sen_max


y_train = y_sensors_sequence[:train_len]
y_val = y_sensors_sequence[train_len:train_len+val_len]
y_test = y_sensors_sequence[train_len+val_len:]


# Reshape X data
X_im_train = np.reshape(X_im_train, newshape = (-1, X_im_train.shape[1], X_im_train.shape[2], X_im_train.shape[3], 1))
X_im_val = np.reshape(X_im_val, newshape = (-1, X_im_val.shape[1], X_im_val.shape[2], X_im_val.shape[3], 1))
X_im_test = np.reshape(X_im_test, newshape = (-1, X_im_test.shape[1], X_im_test.shape[2], X_im_test.shape[3], 1))

In [ ]:
sequence_length = X_im_train.shape[1]
image_height = X_im_train.shape[2]
image_width = X_im_train.shape[3]
num_channels = X_im_train.shape[4]

# Input for image sequences
image_input = Input(shape=(sequence_length, image_height, image_width, num_channels))

# Input for scalar sequences
scalar_input = Input(shape=(sequence_length, 2))

# Define the image processing branch
image_branch = TimeDistributed(Conv2D(32, (3, 3), activation='relu'))(image_input)
image_branch = TimeDistributed(MaxPooling2D((2, 2)))(image_branch)
image_branch = TimeDistributed(Flatten())(image_branch)
image_branch = LSTM(16)(image_branch)

# Define the scalar sequence processing branch
scalar_branch = LSTM(16)(scalar_input)

# Concatenate the outputs of the two branches
concatenated = Concatenate()([image_branch, scalar_branch])

# Fully connected layers for further processing
# dense_layer = Dense(128, activation='relu')(concatenated)
output = Dense(2, activation='linear')(concatenated)

# Create the model
model = Model(inputs=[image_input, scalar_input], outputs=output)

# Compile the model
model.compile(optimizer='Adam', loss='mean_squared_error', metrics=['mae'])

# Print a summary of the model architecture
model.summary()

In [ ]:
history = model.fit(x=[X_im_train, X_sen_train], y=y_train, epochs=1000, batch_size=256, validation_data=([X_im_val, X_sen_val], y_val))

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
y_hat_train = model.predict([X_im_train, X_sen_train])

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# For the first column
axes[0].plot(y_hat_train[:, 0], label='Predicted 0')
axes[0].plot(y_train[:, 0], label='Reference 0', alpha=.5)
axes[0].legend()
axes[0].set_title('Margarethenklippe_Pegel_now')

# For the second column
axes[1].plot(y_hat_train[:, 1], label='Predicted 1')
axes[1].plot(y_train[:, 1], label='Reference 1', alpha=.5)
axes[1].legend()
axes[1].set_title('Sennhuette_Pegel_now')

plt.show()


In [ ]:
y_hat_val = model.predict([X_im_val, X_sen_val])

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# For the first column
axes[0].plot(y_hat_train[:, 0], label='Predicted 0')
axes[0].plot(y_train[:, 0], label='Reference 0', alpha=.5)
axes[0].legend()
axes[0].set_title('Margarethenklippe_Pegel_now')

# For the second column
axes[1].plot(y_hat_train[:, 1], label='Predicted 1')
axes[1].plot(y_train[:, 1], label='Reference 1', alpha=.5)
axes[1].legend()
axes[1].set_title('Sennhuette_Pegel_now')

plt.show()


In [ ]:
y_hat_test = model.predict([X_im_test, X_sen_test])

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# For the first column
axes[0].plot(y_hat_train[:, 0], label='Predicted 0')
axes[0].plot(y_train[:, 0], label='Reference 0', alpha=.5)
axes[0].legend()
axes[0].set_title('Margarethenklippe_Pegel_now')

# For the second column
axes[1].plot(y_hat_train[:, 1], label='Predicted 1')
axes[1].plot(y_train[:, 1], label='Reference 1', alpha=.5)
axes[1].legend()
axes[1].set_title('Sennhuette_Pegel_now')

plt.show()



In [ ]:

# Define the metrics
def mse(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

def mae(y_true, y_pred):
    return np.abs(y_true - y_pred).mean()

def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true)))

def nse(y_true, y_pred):
    return 1 - (np.sum((y_true - y_pred) ** 2) / np.sum((y_true - y_true.mean()) ** 2))

def mbd(y_true, y_pred):
    return np.mean(y_pred - y_true)

datasets = {
    'train': (y_train, y_hat_train),
    'test': (y_test, y_hat_test),
    'val': (y_val, y_hat_val)
}

for name, (y_true, y_pred) in datasets.items():
    print(f"Metrics for {name} dataset:")
    print(f"MSE: {mse(y_true, y_pred)}")
    print(f"RMSE: {rmse(y_true, y_pred)}")
    print(f"MAE: {mae(y_true, y_pred)}")
    print(f"SMAPE: {smape(y_true, y_pred)}")
    print(f"NSE: {nse(y_true, y_pred)}")
    print(f"MBD: {mbd(y_true, y_pred)}")
    print("-" * 30)


In [ ]:
# List of column names
column_names = ['Margarethenklippe_Pegel_now','Sennhuette_Pegel_now']

# Function to plot residuals
def plot_residuals(y_true, y_pred, column_name):
    residuals = y_true - y_pred
    plt.scatter(y_pred, residuals, alpha=0.5)
    plt.axhline(0, color='r', linestyle='--')
    plt.title(f"Residual Plot for {column_name}")
    plt.xlabel(f"Predicted Values for {column_name}")
    plt.ylabel("Residuals")
    plt.show()

# Calculate residuals for each column
for i in range(y_test.shape[1]):
    y_true_column = y_test[:, i]
    y_pred_column = y_hat_test[:, i]
    
    plot_residuals(y_true_column, y_pred_column, column_names[i])

In [ ]:
Die Projektionsebene schneidet die Erdkugel bei 60,0°N
(ϕ0) 5. Das kartesische Koordinatensystem besitzt eine Größe von 900 km x 900 km und ist parallel
zum 10,0°E-Meridian (λ0) ausgerichtet. Als Bezugspunkt wurde der Mittelpunkt des Komposnp.random.seed(42)  # For reproducibility
custom_index = 200  
sequence_length = 8 #2H

# Actual and predicted values for the sequence
actual_sequence = y_test[custom_index:custom_index + sequence_length]
predicted_sequence = y_hat_test[custom_index:custom_index + sequence_length]

# Plotting the selected sequence for both columns
column_names = ['Margarethenklippe_Pegel_now','Sennhuette_Pegel_now']

for i in range(2):
    plt.figure(figsize=(10, 4))
    plt.plot(actual_sequence[:, i], label='Actual', marker='o')
    plt.plot(predicted_sequence[:, i], label='Predicted', marker='x')
    plt.title(f"Actual vs Predicted for {column_names[i]}")
    plt.xlabel("Time step")
    plt.ylabel(column_names[i])
    plt.legend()
    plt.show()

In [ ]:
np.random.seed(42)  # For reproducibility
custom_index = 200  
sequence_length = 24 #6H

# Actual and predicted values for the sequence
actual_sequence = y_test[custom_index:custom_index + sequence_length]
predicted_sequence = y_hat_test[custom_index:custom_index + sequence_length]

# Plotting the selected sequence for both columns
column_names = ['Margarethenklippe_Pegel_now','Sennhuette_Pegel_now']

for i in range(2):
    plt.figure(figsize=(10, 4))
    plt.plot(actual_sequence[:, i], label='Actual', marker='o')
    plt.plot(predicted_sequence[:, i], label='Predicted', marker='x')
    plt.title(f"Actual vs Predicted for {column_names[i]}")
    plt.xlabel("Time step")
    plt.ylabel(column_names[i])
    plt.legend()
    plt.show()